In [ ]:
'''
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

from functools import reduce
import re
import collections
import nltk
#nltk.download('stopwords')
#nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
stopword = set(stopwords.words('english'))

# specify GPU
'''

In [1]:
!pip install transformers==2.3.0

In [2]:
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
df = pd.read_csv('Covid_Papers.csv')
df.drop("Unnamed: 0", axis =1, inplace = True)
df['abstract'] = df['title'] + ' ' + df['abstract']
df.drop("title", axis =1, inplace = True)

In [4]:
def conv(val):
    val = val.replace("'","")
    val = val.strip('][').split(', ')
    return val

In [5]:
df['tags'] = df['tags'].apply(conv)

In [6]:
tags_list = []
for i in range(len(df)):
    tags_list = tags_list + df['tags'][i]
tags_list = list(set(tags_list))

In [7]:
df['tags'] = [list(filter(None, df['tags'][i])) for i in range(len(df))]
df = df[df['tags'].map(lambda d: len(d)) > 0]
df.reset_index(inplace = True, drop = True)

In [8]:
df = df.reindex(columns = df.columns.tolist() + tags_list, fill_value = 0)

In [9]:
for a in range(len(df)):
    for tag in (df['tags'][a]):
        df.at[a,tag] = 1
df.drop("tags", axis =1, inplace = True)

In [10]:
df

,abstract,Hemobilia / etiology*,Computer-Assisted* / methods,Non-alcoholic Fatty Liver Disease / complications,Ulcerative / therapy*,Cognitive Dysfunction* / virology,Hearing Loss* / etiology,Adenosine Deaminase / immunology,Respiration Disorders / etiology,Catalytic Domain,...,Pandemics / statistics &amp; numerical data*,Membrane Proteins / metabolism,Adenosine / genetics,Respiratory Syncytial Virus Infections / mortality,Interferon Type I / genetics,Acute Lung Injury / drug therapy*,Human / epidemiology,Female / therapy,Drug Delivery Systems / methods*,Anorexia Nervosa* / epidemiology
0,Real-World Experience with COVID-19 Including...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Successful outcome of pre-engraftment COVID-19...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The impact of COVID-19 on oncology professiona...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ICU admission and mortality classifiers for CO...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Clinical evaluation of nasopharyngeal midturb...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5468,Hypersensitivity Reactions to Vaccines Curren...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5469,Rooming-in Breastfeeding and Neonatal Follow-...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5470,Acute Abducens Nerve Palsy Following the Secon...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5471,Planning and Implementing the Protocol for Psy...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df.to_csv("IwannaCry.csv")

In [11]:
from sklearn.model_selection import train_test_split

test_split = 0.1

# Initial train and test split.
df_train, df_test = train_test_split(df, test_size=test_split)

# Splitting the test set further into validation
# and new test sets.
df_val = df_test.sample(frac=0.5)
df_test.drop(df_val.index, inplace=True)

D:\Programming\Anaconda\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [12]:
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences

bert_model_name = 'bert-base-uncased'

tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True)
MAX_LEN = 128

def tokenize_sentences(sentences, tokenizer, max_seq_len = 128):
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        tokenized_sentence = tokenizer.encode(
                            sentence,                  # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_seq_len,  # Truncate all sentences.
                    )
        
        tokenized_sentences.append(tokenized_sentence)

    return tokenized_sentences

def create_attention_masks(tokenized_and_padded_sentences):
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        att_mask = [int(token_id > 0) for token_id in sentence]
        attention_masks.append(att_mask)

    return np.asarray(attention_masks)

input_ids = tokenize_sentences(df_train['abstract'], tokenizer, MAX_LEN)
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
attention_masks = create_attention_masks(input_ids)

  0%|          | 0/90 [00:00<?, ?it/s]

In [13]:
label_cols = df_train.columns[1:]
labels =  df_train[label_cols].values

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=0, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, random_state=0, test_size=0.1)

train_size = len(df_train)
validation_size = len(df_val)

In [14]:
BATCH_SIZE = 32
NR_EPOCHS = 1

def create_dataset(data_tuple, epochs=1, batch_size=32, buffer_size=10000, train=True):
    dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
    if train:
        dataset = dataset.shuffle(buffer_size=buffer_size)
    dataset = dataset.repeat(epochs)
    dataset = dataset.batch(batch_size)
    if train:
        dataset = dataset.prefetch(1)
    
    return dataset

train_dataset = create_dataset((train_inputs, train_masks, train_labels), epochs=NR_EPOCHS, batch_size=BATCH_SIZE)
validation_dataset = create_dataset((validation_inputs, validation_masks, validation_labels), epochs=NR_EPOCHS, batch_size=BATCH_SIZE)

In [15]:
len(tf.config.list_physical_devices('GPU'))

1

In [16]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [17]:
pip install tokenization

Note: you may need to restart the kernel to use updated packages.


In [18]:
import tensorflow_hub as hub
import tokenization
module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(module_url, trainable=True)

INFO:absl:Using C:\Users\moaaz\AppData\Local\Temp\tfhub_modules to cache modules.


In [19]:
from transformers import TFBertModel
from tensorflow.keras.layers import Dense, Flatten

class BertClassifier(tf.keras.Model):    
    def __init__(self, bert: TFBertModel, num_classes: int):
        super().__init__()
        self.bert = bert
        self.classifier = Dense(num_classes, activation='sigmoid')
        
    @tf.function
    def call(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
        outputs = self.bert(input_ids,
                               attention_mask=attention_mask,
                               token_type_ids=token_type_ids,
                               position_ids=position_ids,
                               head_mask=head_mask)
        cls_output = outputs[1]
        cls_output = self.classifier(cls_output)
                
        return cls_output

model = BertClassifier(TFBertModel.from_pretrained(bert_model_name), len(label_cols))

In [20]:
import time
from transformers import create_optimizer

steps_per_epoch = train_size // BATCH_SIZE
validation_steps = validation_size // BATCH_SIZE

# | Loss Function
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=False)
train_loss = tf.keras.metrics.Mean(name='train_loss')
validation_loss = tf.keras.metrics.Mean(name='test_loss')

# | Optimizer (with 1-cycle-policy)
warmup_steps = steps_per_epoch // 3
total_steps = steps_per_epoch * NR_EPOCHS - warmup_steps
optimizer = create_optimizer(init_lr=2e-5, num_train_steps=total_steps, num_warmup_steps=warmup_steps)

# | Metrics
train_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]
validation_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]



In [21]:
@tf.function
def train_step(model, token_ids, masks, labels):
    labels = tf.dtypes.cast(labels, tf.float32)

    with tf.GradientTape() as tape:
        predictions = model(token_ids, attention_mask=masks)
        loss = loss_object(labels, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables), 1.0)

    train_loss(loss)

    for i, auc in enumerate(train_auc_metrics):
        auc.update_state(labels[:,i], predictions[:,i])

In [22]:
@tf.function
def validation_step(model, token_ids, masks, labels):
    labels = tf.dtypes.cast(labels, tf.float32)

    predictions = model(token_ids, attention_mask=masks, training=False)
    v_loss = loss_object(labels, predictions)

    validation_loss(v_loss)
    for i, auc in enumerate(validation_auc_metrics):
        auc.update_state(labels[:,i], predictions[:,i])

In [23]:
def train(model, train_dataset, val_dataset, train_steps_per_epoch, val_steps_per_epoch, epochs):
    for epoch in range(epochs):
        print('=' * 50, f"EPOCH {epoch}", '=' * 50)

        start = time.time()

        for i, (token_ids, masks, labels) in enumerate(tqdm(train_dataset, total=train_steps_per_epoch)):
            train_step(model, token_ids, masks, labels)
            if i % 1000 == 0:
                print(f'\nTrain Step: {i}, Loss: {train_loss.result()}')
                for i, label_name in enumerate(label_cols):
                    print(f"{label_name} roc_auc {train_auc_metrics[i].result()}")
                    train_auc_metrics[i].reset_states()
        
        for i, (token_ids, masks, labels) in enumerate(tqdm(val_dataset, total=val_steps_per_epoch)):
            validation_step(model, token_ids, masks, labels)

        print(f'\nEpoch {epoch+1}, Validation Loss: {validation_loss.result()}, Time: {time.time()-start}\n')

        for i, label_name in enumerate(label_cols):
            print(f"{label_name} roc_auc {validation_auc_metrics[i].result()}")
            validation_auc_metrics[i].reset_states()

        print('\n')

In [24]:
train(model, train_dataset, validation_dataset, train_steps_per_epoch=steps_per_epoch, val_steps_per_epoch=validation_steps, epochs=NR_EPOCHS)

================================================== EPOCH 0 ==================================================


  0%|          | 0/2 [00:00<?, ?it/s]

ResourceExhaustedError: Graph execution error:

Detected at node 'tf_bert_model/bert/encoder/layer_._1/intermediate/activation/mul_1' defined at (most recent call last):
    File "D:\Programming\Anaconda\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "D:\Programming\Anaconda\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "D:\Programming\Anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "D:\Programming\Anaconda\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "D:\Programming\Anaconda\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "D:\Programming\Anaconda\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "D:\Programming\Anaconda\lib\asyncio\base_events.py", line 596, i

In [ ]:
test_input_ids = tokenize_sentences(df_test['abstract'], tokenizer, MAX_LEN)
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
test_attention_masks = create_attention_masks(test_input_ids)

In [ ]:
TEST_BATCH_SIZE = 32
test_steps = len(df_test) // TEST_BATCH_SIZE

test_dataset = create_dataset((test_input_ids, test_attention_masks), batch_size=TEST_BATCH_SIZE, train=False, epochs=1)

df_pred = pd.read_csv(subm_path, index_col='id')

for i, (token_ids, masks) in enumerate(tqdm(test_dataset, total=test_steps)):
    sample_ids = df_test.iloc[i*TEST_BATCH_SIZE:(i+1)*TEST_BATCH_SIZE]['id']
    predictions = model(token_ids, attention_mask=masks).numpy()

    df_pred.loc[sample_ids, label_cols] = predictions

In [ ]:
def remove_punctuation(text):
    
    # punctuations except -  
    punc ='''?!.,:;_—[](){}'"`~|\/@#$%^&+=*'''
    for i in text:
        if i in punc:
            text = text.replace(i, ' ')            
    return text.strip()

def preprocess(text):
    
    # lower casing
    text=text.lower()
    
    # stopword removal
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    
    # lemmatization
    text = [lemmatizer.lemmatize(word) for word in text.split(' ')]
    text = " ".join(text)
    
    # removing words containing only numbers
    text = re.sub(r'\s[0-9]+\s', '', text)
    
    # remove extra spaces
    text = re.sub("\s\s+", " ", text)   
    return text.strip()

In [ ]:
df['title']=df['title'].apply(remove_punctuation)
df['title']=df['title'].apply(preprocess)

df['abstract']=df['abstract'].apply(remove_punctuation)
df['abstract']=df['abstract'].apply(preprocess)

In [ ]:
def conv(val):
    val = val.replace("'","")
    val = val.strip('][').split(', ')
    return val

In [ ]:
df['tags'] = df['tags'].apply(conv)

In [ ]:
df['abstract'] = df['title'] + ' ' + df['abstract']

In [ ]:
# Make sure there is no spaces in the empty elements. Example: ' ' -> ''
for i in range(len(df)):
    for j, word in enumerate(df['tags'][i]):
        df['tags'][i][j] = df['tags'][i][j].strip()
        
# Mark None for the the keyword, covid-19 and humans, since almost all the abstracts have as keyword.
# Mark None for the empty elements '' 
for i in range(len(df)):
    for j, word in enumerate(df['tags'][i]):
        if word == '':
            df['tags'][i][j] = None
            
df['tags'] = [list(filter(None, df['tags'][i])) for i in range(len(df))]

In [ ]:
# Generate a list of all the tags 
tags_list = []
for i in range(len(df)):
    tags_list = tags_list + df['tags'][i]

# Counts for each unique keyword
counter=collections.Counter(tags_list)
print('Number of total unique tags: {}'.format(len(counter)))

# Pull the 20 most common words
most_common_words= [word for word, word_count in collections.Counter(tags_list).most_common(20)]
print('20 most common tags: {}'.format(most_common_words))

In [ ]:
# Keep only the most common tags and if abstract doesn't include any of the common tags, remove completely
for i in range(len(df)):
    for j, word in enumerate(df['tags'][i]):
        if word not in most_common_words:
            df['tags'][i][j] = None
                 
df['tags'] = [list(filter(None, df['tags'][i])) for i in range(len(df))]
df = df[df['tags'].map(lambda d: len(d)) > 0]
df.reset_index(inplace = True, drop = True)

print('Number of total abstracts after removal: {}'.format(len(df)))

In [ ]:
tags_list = list(set(tags_list))

In [ ]:
df.drop("title", axis =1, inplace = True)

In [ ]:
df = df.reindex(columns = df.columns.tolist() + tags_list, fill_value = 0)

In [ ]:
df

In [ ]:
for a in range(len(df)):
    for tag in (df['tags'][a]):
        df.at[a,tag] = 1

In [ ]:
df

In [ ]:
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)